In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa

from dataloader.utils import get_count_classes, plot_classes, plot_from_one_class

SEED = 0

# Load datasets

In [2]:
ds_train, ds_val = tf.keras.utils.image_dataset_from_directory(
    directory="tiny-imagenet-200/train/",
    batch_size=128,
    image_size=(64, 64),
    shuffle=True,
    seed=SEED,
    validation_split=0.2,
    subset="both"
    )

Found 100000 files belonging to 200 classes.
Using 80000 files for training.
Using 20000 files for validation.
Metal device set to: Apple M1


2022-12-21 09:16:07.014243: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-21 09:16:07.014388: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Load model

In [3]:
feature_layer = hub.KerasLayer("https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2")

model = tf.keras.Sequential([
    feature_layer,
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings
])

In [4]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

In [5]:
# Train the network
history = model.fit(
    ds_train,
    epochs=1)

2022-12-21 09:16:23.958997: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-21 09:16:23.995235: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 98s 145ms/step - loss: 0.9958


# Evaluate model

In [6]:
model.predict(ds_val)

2022-12-21 09:17:56.342222: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


157/157 [==============================] - 24s 146ms/step


array([[-7.5757998e-05, -4.5162878e-09, -1.4489256e-25, ...,
        -8.3673528e-21, -4.1457671e-12, -1.4790303e-04],
       [-1.0738067e-31, -1.4747615e-06,  0.0000000e+00, ...,
        -4.1538949e-20,  2.4478859e-03,  3.0744132e-02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00, -2.6299375e-17],
       ...,
       [-3.3578297e-06,  4.8777264e-02, -1.5114010e-06, ...,
        -4.2849418e-12, -3.8702380e-13, -2.8337383e-11],
       [-7.9475321e-05, -1.7508333e-04, -1.1933424e-26, ...,
        -4.6231605e-15, -1.7810679e-07, -5.1165596e-08],
       [-2.4036795e-12,  1.0545861e-02,  0.0000000e+00, ...,
        -4.0958357e-28,  0.0000000e+00, -4.3341282e-36]], dtype=float32)

In [7]:
model.evaluate(ds_val)

2022-12-21 09:18:20.713197: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


157/157 [==============================] - 24s 142ms/step - loss: 0.9959


0.9959089159965515

In [8]:
ds_one_class = ds_val.unbatch().filter(lambda x, y: y==100)

In [10]:
model.evaluate(ds_one_class.batch(32))

3/3 [==============================] - 5s 2s/step - loss: 2.1930


2.193035840988159